In [1]:
# install required libraries
# !pip install pandas

In [2]:
# import required libraries
import os

import pandas as pd

In [3]:
# set global variables
DATA_DIR = "../data"

In [4]:
# load the dataset
df = pd.read_csv(os.path.join(DATA_DIR, "train_data.csv"))
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,Cash loans,F,N,Y,1,67500.0,227520.0,11065.5,180000.0,...,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,Cash loans,M,Y,Y,1,247500.0,1882372.5,65560.5,1719000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,Cash loans,M,Y,Y,1,180000.0,101880.0,10827.0,90000.0,...,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0,0
3,305311,Cash loans,M,N,N,0,81000.0,405000.0,20677.5,405000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0
4,414121,Cash loans,F,N,Y,0,157500.0,888840.0,29506.5,675000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0


Above we can see a sample of the dataset. Here for each `SK_ID_CURR` we need to predict the `TARGET` column.

Let's look at data more deeply.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184506 entries, 0 to 184505
Columns: 122 entries, SK_ID_CURR to TARGET
dtypes: float64(65), int64(41), object(16)
memory usage: 171.7+ MB


We can observer from the output of above cell that:  
- There are `1,84,506` data points in total
- There are `122` columns, including the target column
- Columns with different datatypes are present:
    - `65` columns of float datatype,
    - `41` columns of integer datatype, and
    - `16` columns of object datatype

Let us check if there are any duplicate rows in the dataset

In [6]:
duplicated_rows = len(df.index[df.duplicated()])
print(f"There are {duplicated_rows} duplicate rows in the dataset.")

There are 0 duplicate rows in the dataset.


As we can see, there are no duplicate rows in the dataset.  
  
Let us see if we have any missing values present in our dataset.

In [7]:
missing_data_info = df.isna().sum()
missing_data_info = missing_data_info[missing_data_info.gt(0)]
print(f"There are {len(missing_data_info)} columns with missing values.\n")

missing_data_info_sorted = dict(sorted(missing_data_info.items(), key=lambda item: item[1]))
for column_name, num_of_null_values in missing_data_info_sorted.items():
    print(column_name, "-", num_of_null_values)

There are 67 columns with missing values.

CNT_FAM_MEMBERS - 1
DAYS_LAST_PHONE_CHANGE - 1
AMT_ANNUITY - 6
AMT_GOODS_PRICE - 167
EXT_SOURCE_2 - 413
OBS_30_CNT_SOCIAL_CIRCLE - 639
DEF_30_CNT_SOCIAL_CIRCLE - 639
OBS_60_CNT_SOCIAL_CIRCLE - 639
DEF_60_CNT_SOCIAL_CIRCLE - 639
NAME_TYPE_SUITE - 770
AMT_REQ_CREDIT_BUREAU_HOUR - 24998
AMT_REQ_CREDIT_BUREAU_DAY - 24998
AMT_REQ_CREDIT_BUREAU_WEEK - 24998
AMT_REQ_CREDIT_BUREAU_MON - 24998
AMT_REQ_CREDIT_BUREAU_QRT - 24998
AMT_REQ_CREDIT_BUREAU_YEAR - 24998
EXT_SOURCE_3 - 36656
OCCUPATION_TYPE - 57867
EMERGENCYSTATE_MODE - 87336
TOTALAREA_MODE - 88948
YEARS_BEGINEXPLUATATION_AVG - 89900
YEARS_BEGINEXPLUATATION_MODE - 89900
YEARS_BEGINEXPLUATATION_MEDI - 89900
FLOORSMAX_AVG - 91726
FLOORSMAX_MODE - 91726
FLOORSMAX_MEDI - 91726
HOUSETYPE_MODE - 92482
LIVINGAREA_AVG - 92534
LIVINGAREA_MODE - 92534
LIVINGAREA_MEDI - 92534
ENTRANCES_AVG - 92807
ENTRANCES_MODE - 92807
ENTRANCES_MEDI - 92807
APARTMENTS_AVG - 93575
APARTMENTS_MODE - 93575
APARTMENTS_MEDI -